In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from envs.GraphEnv.impnode import ImpnodeEnv
from envs.sync_vector_env import GraphSyncVectorEnv
from envs.async_vector_env import GraphAsyncVectorEnv
from envs.async_vector_env2 import GraphAsyncVectorEnv2
import  numpy as np
import time

In [3]:
def make_env_fn(seed):
    env = ImpnodeEnv(ba_nodes = 10, ba_edges = 6, max_removed_nodes = 5, seed = seed)
    return env

num_envs = 10
num_episodes = 1

In [4]:
%load_ext line_profiler

In [5]:
def sync_func(num_envs, num_episodes):
    sync_envs = GraphSyncVectorEnv([make_env_fn(i) for i in range(num_envs)])
    for i in range(num_envs):
        sync_envs.action_space(i).seed(i)

    for i in range(num_episodes):
        terminated = False
        num_nodes = [sync_envs.num_nodes(i) for i in range(num_envs)]
        masks = [np.ones(nodes, dtype=np.int8) for nodes in num_nodes]
        while not terminated:
            actions = [sync_envs.action_space(i).sample(mask=masks[i]) for i in range(num_envs)]
            for j in range(num_envs):
                masks[j][actions[j]] = 0
            start = time.time()
            observation, reward, terminated, truncated, info = sync_envs.step(actions)
            end = time.time()
            print(end-start)
            terminated = sum(terminated)
        sync_envs.reset()


In [6]:
sync_func(num_envs, num_episodes)

1.0938084125518799
1.0901577472686768
1.0885083675384521
1.0909719467163086
1.0932607650756836


In [7]:
def async_func(num_envs, num_episodes):
    async_envs = GraphAsyncVectorEnv([make_env_fn(i) for i in range(num_envs)])
    for i in range(num_envs):
        async_envs.action_space(i).seed(i)
    if __name__ == '__main__':
        for i in range(num_episodes):
            terminated = False
            num_nodes = [async_envs.num_nodes(i) for i in range(num_envs)]
            masks = [np.ones(nodes, dtype=np.int8) for nodes in num_nodes]
            while not terminated:
                actions = [async_envs.action_space(i).sample(mask=masks[i]) for i in range(num_envs)]
                for j in range(num_envs):
                    masks[j][actions[j]] = 0
                start = time.time()
                observation, reward, terminated, truncated, info = async_envs.step(actions)
                end = time.time()
                print(end-start)
                terminated = sum(terminated)
            async_envs.reset()


In [8]:
async_func(num_envs, num_episodes)

0.2327280044555664
0.2301652431488037
0.23233675956726074
0.23167824745178223
0.21840667724609375
